In [157]:
%load_ext autoreload
%autoreload 2

import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
from sklearn.preprocessing import MinMaxScaler
import xarray as xr
#import pymaise 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [158]:
from pyMAISE.datasets import load_anomaly
from pyMAISE.preprocessing import train_test_split, scale_data, one_hot_encode, SplitSequence
import pyMAISE as mai

In [159]:
global_settings = mai.init(
    problem_type=mai.ProblemType.CLASSIFICATION,   # Define a regression problem
    cuda_visible_devices="-1"                  # Use CPU only/ Delete line when run on GPU
)

In [160]:
# call handler
input_path = "./../../../../DTL.npy"
output_path = "./../../../../DTL_labels.npy"
#print(mai.__file__)

inputs, outputs = load_anomaly([input_path, output_path], True, True, True)


In [161]:
inputs = np.array(inputs) #add names for 14 from paper
inputs

array([[-6.07100e+00,  5.55120e+00, -1.42220e+00, ..., -4.60370e-02,
         2.17170e+03,  1.27100e+00],
       [-4.85030e+00,  4.33040e+00, -2.64290e+00, ...,  1.50000e-02,
         2.17250e+03, -1.33776e+01],
       [-4.85030e+00,  1.88890e+00, -1.42220e+00, ...,  4.55190e-02,
         2.17320e+03, -8.49500e+00],
       ...,
       [-2.17470e+00, -7.57620e+00, -3.39550e+00, ..., -1.07070e-01,
         2.17220e+03,  1.19730e+01],
       [-4.61620e+00, -3.91400e+00, -5.83700e+00, ..., -1.68110e-01,
         2.17180e+03,  9.53200e+00],
       [-3.39550e+00, -6.35550e+00, -5.83700e+00, ..., -4.60370e-02,
         2.17150e+03,  9.53200e+00]], dtype=float32)

In [162]:
outputs = np.array(outputs)
outputs



array([['Normal'],
       ['Normal'],
       ['Normal'],
       ...,
       ['DV/DT Fault'],
       ['DV/DT Fault'],
       ['DV/DT Fault']], dtype=object)

In [163]:
column_names_in = ['A+IGBT-I: current', 
                'A+*IGBT-I: current', 
                'B+IGBT-I: current', 
                'B+*IGBT-I: current', 
                'C+IGBT-I: current', 
                'C+*IGBT-I: current', 
                'A-Flux', 
                'B-Flux', 
                'C-Flux', 
                'Mod-V', 
                'Mod-I', 
                'CB-I', 
                'CB-V', 
                'DV/DT']
inp = xr.DataArray(
    inputs,
    coords={"lon": np.arange(inputs.shape[0]), "col": column_names_in},
    dims=("lon", "col"),
)
#inp.col.attrs["name"] = "columns"
#inputs.shape[0]
inp


<xarray.DataArray (lon: 4846500, col: 14)>
array([[-6.07100e+00,  5.55120e+00, -1.42220e+00, ..., -4.60370e-02,
         2.17170e+03,  1.27100e+00],
       [-4.85030e+00,  4.33040e+00, -2.64290e+00, ...,  1.50000e-02,
         2.17250e+03, -1.33776e+01],
       [-4.85030e+00,  1.88890e+00, -1.42220e+00, ...,  4.55190e-02,
         2.17320e+03, -8.49500e+00],
       ...,
       [-2.17470e+00, -7.57620e+00, -3.39550e+00, ..., -1.07070e-01,
         2.17220e+03,  1.19730e+01],
       [-4.61620e+00, -3.91400e+00, -5.83700e+00, ..., -1.68110e-01,
         2.17180e+03,  9.53200e+00],
       [-3.39550e+00, -6.35550e+00, -5.83700e+00, ..., -4.60370e-02,
         2.17150e+03,  9.53200e+00]], dtype=float32)
Coordinates:
  * lon      (lon) int64 0 1 2 3 4 5 ... 4846495 4846496 4846497 4846498 4846499
  * col      (col) <U18 'A+IGBT-I: current' 'A+*IGBT-I: current' ... 'DV/DT'

In [164]:
column_names_out = ['Fault Status']
out = xr.DataArray(
    outputs,
    coords={"lon": np.arange(inputs.shape[0]), "col": column_names_out},
    dims=("lon", "col"),
)
#out.col.attrs["name"] = "columns"

out


<xarray.DataArray (lon: 4846500, col: 1)>
array([['Normal'],
       ['Normal'],
       ['Normal'],
       ...,
       ['DV/DT Fault'],
       ['DV/DT Fault'],
       ['DV/DT Fault']], dtype=object)
Coordinates:
  * lon      (lon) int64 0 1 2 3 4 5 ... 4846495 4846496 4846497 4846498 4846499
  * col      (col) <U12 'Fault Status'

In [165]:
import xarray as xr
from pyMAISE.preprocessing import one_hot_encode

# One-hot encode input data
#encoded_inputs = one_hot_encode(inp)

# One-hot encode output data
encoded_outputs = one_hot_encode(out)



#print("One-hot encoded inputs:")
#print(encoded_inputs)

print("One-hot encoded outputs:")
print(encoded_outputs)



One-hot encoded outputs:
<xarray.DataArray (lon: 4846500, col: 7)>
array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * lon      (lon) int64 0 1 2 3 4 5 ... 4846495 4846496 4846497 4846498 4846499
  * col      (col) object 'Fault Status_DV/DT Fault' ... 'Fault Status_SCR AC...


In [166]:
#TESTING SPLIT SEQUENCE
from pyMAISE import datasets, preprocessing
#_, perturbed = datasets.load_loca(stack_series=True)
#perturbed.shape
#perturbed


In [167]:
#perturbed.shape

In [168]:
inp.shape

(4846500, 14)

In [169]:
sequence_outputs = [
                'A+IGBT-I: current', 
                'A+*IGBT-I: current', 
                'B+IGBT-I: current', 
                'B+*IGBT-I: current', 
                'C+IGBT-I: current', 
                'C+*IGBT-I: current', 
                'A-Flux', 
                'B-Flux', 
                'C-Flux', 
                'Mod-V', 
                'Mod-I', 
                'CB-I', 
                'CB-V', 
                'DV/DT'
]
split_sequences = preprocessing.SplitSequence(
        5,
        1,
        1,
        sequence_outputs=sequence_outputs
)
perturbed_input, perturbed_output = split_sequences.split(inp)
perturbed_input.shape



(4846495, 5, 14)

In [170]:
perturbed_output.shape

(4846495, 14)

In [171]:
# one hot 
#split sequence 
 # scale data the inputs 
    
    # all of preprocessing
    
    
#initialize ML methods 
#